## Computational Theory Assessment

**Author:** Viktor Kobylka G00425163 
**Module:** COMPUTATIONAL THEORY  
**Institution:** ATU  

### Introduction

This notebook presents an implementation of key components of the Secure Hash
Standard (SHA-256). The aim of this assessment is to demonstrate an understanding of:
1. Binary word operations (bitwise functions)
2. Constant generation from prime numbers
3. Message padding scheme
4. Compression function
5. Security analysis through password cracking

In [29]:
import numpy as np
import os

# Hide overflow warnings
np.seterr(over='ignore')

{'divide': 'warn', 'over': 'ignore', 'under': 'ignore', 'invalid': 'warn'}

## Problem 1: Binary Words and Operations


### Overview

SHA-256 uses seven fundamental bitwise operations defined in [Section 4.1.2 of the Secure Hash Standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf). These operations manipulate 32-bit words and form the building blocks of the compression function.

All operations use **32-bit unsigned integers** to ensure consistent behavior across platforms.

### The Seven Operations

1. **Parity(x, y, z)**: XOR operation on three inputs
2. **Ch(x, y, z)**: "Choose" function - selects bits from y or z based on x
3. **Maj(x, y, z)**: "Majority" function - returns majority bit value
4. **Σ₀(x)**: Used in compression function 
5. **Σ₁(x)**: Used in compression function 
6. **σ₀(x)**: Used in message schedule 
7. **σ₁(x)**: Used in message schedule 

#### Parity Function

The Parity function performs a bitwise XOR on three 32-bit words.

**Formula:** `Parity(x, y, z) = x ⊕ y ⊕ z`

The XOR operation returns 1 when an odd number of input bits are 1.

In [30]:
def Parity(x, y, z):
    """
    Calculate parity using XOR: x ⊕ y ⊕ z
    
    Args:
        x (np.uint32): First 32-bit word
        y (np.uint32): Second 32-bit word
        z (np.uint32): Third 32-bit word
    
    Returns:
        np.uint32: Result of x XOR y XOR z
    """
    return np.uint32(x ^ y ^ z)

#### Ch (Choose) Function

The Ch function chooses bits from y or z based on the bits in x.

**Formula:** `Ch(x, y, z) = (x ∧ y) ⊕ (¬x ∧ z)`

**How it works:**
When a bit in x is 1, the corresponding bit from y is chosen.
When a bit in x is 0, the corresponding bit from z is chosen.
This creates a conditional selection based on the control value x.

In [31]:
def Ch(x, y, z):
    """
    Choose function: (x AND y) XOR (NOT x AND z)
    Args:
        x (np.uint32): Selector 32-bit word
        y (np.uint32): First choice 32-bit word
        z (np.uint32): Second choice 32-bit word
    
    Returns:
        np.uint32: Result of (x AND y) XOR (NOT x AND z)
    """
    # (x & y) gives us bits from y where x is 1
    # (~x & z) gives us bits from z where x is 0  
    # XOR combines them into the final result
    return np.uint32((x & y) ^ (~x & z))

#### Maj (Majority) Function

The Maj function returns the majority bit value for each position across three inputs.

**Formula:** `Maj(x, y, z) = (x ∧ y) ⊕ (x ∧ z) ⊕ (y ∧ z)`

**How it works:**
For each bit position, if at least two of the three inputs have a 1, the output is 1.

**Example:**
- x=1, y=1, z=0 -> output=1 (majority is 1)
- x=1, y=0, z=0 -> output=0 (majority is 0)

In [32]:
def Maj(x, y, z):
    """
    Majority function: (x AND y) XOR (x AND z) XOR (y AND z)
    
    Args:
        x (np.uint32): First 32-bit word
        y (np.uint32): Second 32-bit word
        z (np.uint32): Third 32-bit word
    
    Returns:
        np.uint32: Result where each bit is the majority of corresponding bits in x, y, z
    """
    return np.uint32((x & y) ^ (x & z) ^ (y & z))

#### ROTR and SHR

These helper functions implement rotation and shift operations used by the Sigma functions.

**ROTR (Rotate Right):** Circular right shift - bits that fall off the right wrap around to the left.

**SHR (Shift Right):** Logical right shift - bits that fall off are lost, zeros fill from the left.

In [33]:
def ROTR(x, n):
    """
    Rotate right by n positions

    Args:
        x (np.uint32): 32-bit word to rotate
        n (int): Number of positions to rotate right (0-31)
    
    Returns:
        np.uint32: x rotated right by n positions
    """
    # Right shift loses n rightmost bits: x >> n
    # Left shift by (32-n) positions the lost bits on the left: x << (32-n)
    # OR combines them: dropped bits appear on the left side
    return np.uint32((x >> n) | (x << (32 - n)))

def SHR(x, n):
    """
    Shift right by n positions
    
    Args:
        x (np.uint32): 32-bit word to shift
        n (int): Number of positions to shift right (0-31)
    
    Returns:
        np.uint32: x shifted right by n positions (zeros fill from left)
    """
    # Simple right shift - bits fall off, zeros fill from left
    return np.uint32(x >> n)

#### Sigma Functions (Uppercase Σ)

These functions are used in the compression function to mix the working variables.

**Σ₀(x):** `ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)`

**Σ₁(x):** `ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)`

The combination of different rotation amounts provides cryptographic diffusion - small changes in input create large changes in output.

In [34]:
def Sigma0(x):
    """
    SHA-256 Σ₀ function: used in the compression function.
    
    Defined as: Σ₀²⁵⁶(x) = ROTR²(x) ⊕ ROTR¹³(x) ⊕ ROTR²²(x)
    
    Args:
        x (np.uint32): 32-bit word
    
    Returns:
        np.uint32: Result of rotation and XOR operations
    """
    # Rotate by 2, 13, and 22 positions
    # XOR all three rotations together
    return np.uint32(ROTR(x, 2) ^ ROTR(x, 13) ^ ROTR(x, 22))

def Sigma1(x):
    """
    SHA-256 Σ₁ function: used in the compression function.
    
    Defined as: Σ₁²⁵⁶(x) = ROTR⁶(x) ⊕ ROTR¹¹(x) ⊕ ROTR²⁵(x)
    
    Args:
        x (np.uint32): 32-bit word
    
    Returns:
        np.uint32: Result of rotation and XOR operations
    """
    # Rotate by 6, 11, and 25 positions  
    # XOR all three rotations together
    return np.uint32(ROTR(x, 6) ^ ROTR(x, 11) ^ ROTR(x, 25))

#### sigma Functions (Lowercase σ)

These functions are used in the message schedule to extend the 16-word message block to 64 words.

**σ₀(x):** `ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)`

**σ₁(x):** `ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)`

The combination of rotation (ROTR) and shift (SHR) operations contributes to diffusion and helps prevent simple inversion of the hash process.

In [35]:
def sigma0(x):
    """
    SHA-256 σ₀ function: used in message schedule.
    
    Defined as: σ₀²⁵⁶(x) = ROTR⁷(x) ⊕ ROTR¹⁸(x) ⊕ SHR³(x)
    
    Args:
        x (np.uint32): 32-bit word
    
    Returns:
        np.uint32: Result of rotation, shift, and XOR operations
    """
    # Two rotations (7, 18) and one shift (3)
    # XOR all three together
    return np.uint32(ROTR(x, 7) ^ ROTR(x, 18) ^ SHR(x, 3))

def sigma1(x):
    """
    SHA-256 σ₁ function: used in message schedule.
    
    Defined as: σ₁²⁵⁶(x) = ROTR¹⁷(x) ⊕ ROTR¹⁹(x) ⊕ SHR¹⁰(x)
    
    Args:
        x (np.uint32): 32-bit word
    
    Returns:
        np.uint32: Result of rotation, shift, and XOR operations
    """
    # Two rotations (17, 19) and one shift (10)
    # XOR all three together
    return np.uint32(ROTR(x, 17) ^ ROTR(x, 19) ^ SHR(x, 10))

#### Testing Problem 1 functions

In [36]:
# Test Problem 1 functions
x = np.uint32(0xAAAAAAAA)
y = np.uint32(0x55555555)
z = np.uint32(0xFFFFFFFF)

print("Testing binary operations:")
print(f"Parity: {Parity(x, y, z):#010x}")
print(f"Ch: {Ch(x, y, z):#010x}")
print(f"Maj: {Maj(x, y, z):#010x}")
print(f"Sigma0: {Sigma0(x):#010x}")
print(f"Sigma1: {Sigma1(x):#010x}")
print(f"sigma0: {sigma0(x):#010x}")
print(f"sigma1: {sigma1(x):#010x}")

Testing binary operations:
Parity: 0x00000000
Ch: 0x55555555
Maj: 0xffffffff
Sigma0: 0x55555555
Sigma1: 0xaaaaaaaa
sigma0: 0xeaaaaaaa
sigma1: 0x002aaaaa


## Problem 2: Fractional Parts of Cube Roots

### Purpose

SHA-256 uses 64 constant values (K₀ through K₆₃) in its compression function. These constants are derived from nature (cube roots of prime numbers) to ensure they have no hidden mathematical properties that could weaken the algorithm.

**Method:**
1. Take the first 64 prime numbers (2, 3, 5, 7, ...)
2. Calculate the cube root of each prime
3. Take the fractional part (the part after the decimal point)
4. Extract the first 32 bits of this fractional part

This process is defined in [Section 4.2.2 of the standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf).

#### Prime Number Generation

The `primes(n)` function uses the [Sieve of Eratosthenes](https://en.wikipedia.org/wiki/Sieve_of_Eratosthenes), an ancient and efficient algorithm for finding prime numbers.

**Algorithm:**
1. Create a list of numbers up to an estimated limit
2. Mark 0 and 1 as non-prime
3. For each prime found, mark all its multiples as composite
4. Collect the first n unmarked (prime) numbers

In [37]:
def primes(n):
    """
    Generate first n prime numbers using Sieve of Eratosthenes
    
    Args:
        n (int): Number of primes to generate
    
    Returns:
        list: First n prime numbers
    """
    if n <= 0:
        return []
    
    # Estimate upper bound for nth prime using approximation
    if n < 6:
        limit = 15
    else:
        import math
        limit = int(n * (math.log(n) + math.log(math.log(n)))) + 100
    
    # Initialize sieve - assume all numbers are prime initially
    sieve = [True] * limit
    sieve[0] = sieve[1] = False
    
    # Mark composites
    for i in range(2, int(limit**0.5) + 1):
        if sieve[i]:
            # Mark all multiples of i as composite
            for j in range(i*i, limit, i):
                sieve[j] = False
    
    # Collect the first n primes
    result = []
    for i in range(limit):
        if sieve[i]:
            result.append(i)
            if len(result) == n:
                break
    
    return result

#### Calculating K Constants

For each of the first 64 primes, we:
1. Calculate ∛p (cube root)
2. Extract fractional part: ∛p - ⌊∛p⌋
3. Multiply by 2³² to shift bits into integer range
4. Take integer part to get first 32 bits
5. Store as uint32

In [38]:
# Generate first 64 primes
first_64_primes = primes(64)

# Calculate K constants from cube roots
K_constants = []

for prime in first_64_primes:
    # Calculate cube root
    cube_root = prime ** (1/3)

    # Extract fractional part (everything after decimal point)
    fractional_part = cube_root - int(cube_root)

    # Get first 32 bits by multiplying by 2^32 and taking integer part
    first_32_bits = int(fractional_part * (2**32))

    # Store as numpy uint32
    K_constants.append(np.uint32(first_32_bits))

print(f"Calculated {len(K_constants)} constants")
print(f"First 8: {[f'{k:#010x}' for k in K_constants[:8]]}")

Calculated 64 constants
First 8: ['0x428a2f98', '0x71374491', '0xb5c0fbcf', '0xe9b5dba5', '0x3956c25b', '0x59f111f1', '0x923f82a4', '0xab1c5ed5']


#### Verification Against the Standard

We verify our calculated constants match the values specified on page 11 of FIPS 180-4 [Section 4.2.2 of the standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf).

This ensures our implementation is correct and follows the standard exactly.

In [39]:
# Constants from the Secure Hash Standard
STANDARD_K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5,
    0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3,
    0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc,
    0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7,
    0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13,
    0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3,
    0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5,
    0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208,
    0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

# Convert to numpy uint32 for comparison
STANDARD_K = [np.uint32(k) for k in STANDARD_K]

# Verify all constants match
all_match = all(calc == std for calc, std in zip(K_constants, STANDARD_K))
print(f"Constants match standard: {all_match}")

Constants match standard: True


## Problem 3: Padding

### Purpose

SHA-256 operates on 512-bit (64-byte) blocks. Messages of any length must be padded to a multiple of 512 bits according to [Section 5.1.1 of the standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf).

### Padding Scheme

The padding consists of three parts:

1. **Append '1' bit**: Add a single 1 bit (represented as byte 0x80 = 10000000 in binary)
2. **Append '0' bits**: Add zeros until the message length ≡ 448 (mod 512), or equivalently 56 (mod 64) bytes
3. **Append length**: Add the original message length as a 64-bit big-endian integer

This ensures the padded message is always a multiple of 512 bits, even for empty messages.

In [40]:
def block_parse(msg):
    """
    Generator function that pads a message and yields 512-bit blocks.
    
    Padding scheme:
    1. Append bit '1' (0x80 byte)
    2. Append '0' bits until length ≡ 56 (mod 64) bytes
    3. Append original message length as 64-bit big-endian integer
    
    Args:
        msg (bytes): Input message to be padded
    
    Yields:
        bytes: 512-bit (64-byte) blocks including padding
    """
    # Store original message length in bits for the length field
    msg_len_bits = len(msg) * 8
    
    # Start with the original message
    padded = bytearray(msg)
    
    # Step 1: Append the '1' bit as byte 0x80 (10000000 in binary)
    padded.append(0x80)
    
    # Step 2: Append '0' bits until length ≡ 56 (mod 64)
    # This leaves exactly 8 bytes for the length field
    while len(padded) % 64 != 56:
        padded.append(0x00)
    
    # Step 3: Append original message length as 64-bit big-endian integer
    # Big-endian means most significant byte first
    padded.extend(msg_len_bits.to_bytes(8, byteorder='big'))
    
    # Yield 64-byte blocks
    for i in range(0, len(padded), 64):
        yield bytes(padded[i:i+64])
    

#### Testing the Padding Function

Test with messages of different lengths to verify correct padding behavior:
- Empty message (0 bytes)
- Short message (3 bytes)
- Message that exactly fills to byte 55
- Message that requires a second block (56+ bytes)

In [41]:
# Test padding
test_messages = [b"", b"abc", b"a" * 55, b"a" * 56]

for msg in test_messages:
    blocks = list(block_parse(msg))
    print(f"Message length: {len(msg):3d} bytes -> {len(blocks)} block(s) of 64 bytes each")

Message length:   0 bytes -> 1 block(s) of 64 bytes each
Message length:   3 bytes -> 1 block(s) of 64 bytes each
Message length:  55 bytes -> 1 block(s) of 64 bytes each
Message length:  56 bytes -> 2 block(s) of 64 bytes each


## Problem 4: Hashes

### Overview

The `hash()` function implements the core SHA-256 compression function described in [Section 6.2.2 of the standard](https://nvlpubs.nist.gov/nistpubs/FIPS/NIST.FIPS.180-4.pdf). This is where the actual cryptographic transformation occurs.

### Algorithm Steps

1. **Message Schedule Preparation** (W[0] to W[63]):
   - First 16 words (W[0]...W[15]) come directly from the message block
   - Remaining 48 words (W[16]...W[63]) are computed using sigma functions
   - Formula: `W[t] = σ₁(W[t-2]) + W[t-7] + σ₀(W[t-15]) + W[t-16]`

2. **Initialize Working Variables**:
   - Eight 32-bit variables (a, b, c, d, e, f, g, h)
   - Set to current hash values (H₀...H₇)

3. **64 Rounds of Compression**:
   - Each round updates the working variables
   - Uses Ch, Maj, Σ₀, Σ₁ functions from Problem 1
   - Mixes in constants K[t] and message schedule W[t]

4. **Add to Current Hash**:
   - Final working variables are added to current hash
   - This ensures each block contributes to the final result

### Initial Hash Values

SHA-256 starts with specific initial hash values (H₀ through H₇). These are the first 32 bits of the fractional parts of the **square roots** (not cube roots) of the first 8 primes (2, 3, 5, 7, 11, 13, 17, 19).

These are defined in Section 5.3.3 of the standard.

In [42]:
# Initial hash values for SHA-256 (from Section 5.3.3)
INITIAL_HASH = [
    np.uint32(0x6a09e667),
    np.uint32(0xbb67ae85),
    np.uint32(0x3c6ef372),
    np.uint32(0xa54ff53a),
    np.uint32(0x510e527f),
    np.uint32(0x9b05688c),
    np.uint32(0x1f83d9ab),
    np.uint32(0x5be0cd19)
]

#### The Compression Function

This function processes one 512-bit block and updates the hash state.

The compression function is the heart of SHA-256, providing:
- **Diffusion**: Small input changes create large output changes
- **Confusion**: Complex relationship between input and output
- **One-way property**: Computationally infeasible to reverse

In [43]:
def hash(current, block):
    """
    SHA-256 compression function

    Args:
        current (list): Current hash value as 8 uint32 values [H0, H1, ..., H7]
        block (bytes): 512-bit (64-byte) message block
    
    Returns:
        list: New hash value as 8 uint32 values
    """
    # Copy current hash to working array
    H = [np.uint32(h) for h in current]
    
    # Prepare message schedule W[0..63]
    W = []
    
    # W[0..15]: Parse block into 16 32-bit big-endian words
    for i in range(16):
        # Extract 4 bytes and convert to uint32 (big-endian)
        word = int.from_bytes(block[i*4:(i+1)*4], byteorder='big')
        W.append(np.uint32(word))
    
    # W[16..63]: Extend using sigma functions
    for t in range(16, 64):
        # W[t] = σ₁(W[t-2]) + W[t-7] + σ₀(W[t-15]) + W[t-16]
        w = np.uint32(sigma1(W[t-2]) + W[t-7] + sigma0(W[t-15]) + W[t-16])
        W.append(w)
    
    # Initialize working variables with current hash
    a, b, c, d, e, f, g, h = H
    
    # Main compression loop - 64 rounds
    for t in range(64):
        # T1 = h + Σ₁(e) + Ch(e,f,g) + K[t] + W[t]
        T1 = np.uint32(h + Sigma1(e) + Ch(e, f, g) + STANDARD_K[t] + W[t])
        
        # T2 = Σ₀(a) + Maj(a,b,c)
        T2 = np.uint32(Sigma0(a) + Maj(a, b, c))
        
        # Update working variables (rotate through)
        h = g
        g = f
        f = e
        e = np.uint32(d + T1)
        d = c
        c = b
        b = a
        a = np.uint32(T1 + T2)
    
    # Add working variables to current hash (modular addition)
    H[0] = np.uint32(H[0] + a)
    H[1] = np.uint32(H[1] + b)
    H[2] = np.uint32(H[2] + c)
    H[3] = np.uint32(H[3] + d)
    H[4] = np.uint32(H[4] + e)
    H[5] = np.uint32(H[5] + f)
    H[6] = np.uint32(H[6] + g)
    H[7] = np.uint32(H[7] + h)
    
    return H


#### Complete SHA-256 Function

This function combines all components to produce the final SHA-256 hash:
1. Starts with initial hash values
2. Pads message using `block_parse()`
3. Processes each block with `hash()`
4. Converts final hash to hexadecimal string

In [44]:
def sha256(message):
    """
    Complete SHA-256 hash function.
    
    Args:
        message (bytes): Message to hash
    
    Returns:
        str: 64-character hexadecimal hash string (256 bits)
    """
    # Start with initial hash values
    current_hash = INITIAL_HASH.copy()
    
    # Process each 512-bit block
    for block in block_parse(message):
        current_hash = hash(current_hash, block)
    
    # Convert to hexadecimal string (8 hex chars per 32-bit word)
    result = ''.join(f'{h:08x}' for h in current_hash)
    return result

#### Testing SHA-256 Implementation

Test against official test vectors from NIST and other sources to verify correctness.

In [45]:
# Test SHA-256 implementation
test_vectors = [
    (b"", "e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855"),
    (b"abc", "ba7816bf8f01cfea414140de5dae2223b00361a396177a9cb410ff61f20015ad"),
    (b"abcdbcdecdefdefgefghfghighijhijkijkljklmklmnlmnomnopnopq",
     "248d6a61d20638b8e5c026930c3e6039a33ce45964ff2167f6ecedd419db06c1")
]

print("Testing SHA-256:")
for msg, expected in test_vectors:
    result = sha256(msg)
    match = result == expected
    print(f"{str(msg)[:30]:30s} -> Match: {match}")

Testing SHA-256:
b''                            -> Match: True
b'abc'                         -> Match: True
b'abcdbcdecdefdefgefghfghighij -> Match: True


## Problem 5: Passwords

### Objective

Given three SHA-256 hashes, determine the original passwords and analyze security vulnerabilities in unsalted password hashing.

### Target Hashes

These are SHA-256 hashes of common passwords that were hashed using:
- Single pass of SHA-256
- UTF-8 encoding
- No salt

1. `5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8`
2. `873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34`
3. `b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342`

In [46]:
# Target password hashes
target_hashes = [
    "5e884898da28047151d0e56f8dc6292773603d0d6aabbdd62a11ef721d1542d8",
    "873ac9ffea4dd04fa719e8920cd6938f0c23cd678af330939cff53c3d2855f34",
    "b03ddf3ca2e714a6548e7495e2a03f5e824eaac9837cd7f159c67b90fb4b7342"
]

#### Attack Method: Dictionary Attack

A dictionary attack is a method of breaking into a password-protected computer or server by systematically entering every word in a dictionary as a password.

**How it works:**

1. **Compile a password list**: Gather common passwords from:
   - Breach databases (e.g., rockyou.txt with 14 million passwords)
   - Common password lists
   - Dictionary words
   - Pattern-based passwords (e.g., keyboard patterns like "qwerty")

2. **Hash each password**: Using the same algorithm (SHA-256 with UTF-8 encoding)

3. **Compare hashes**: Check if any computed hash matches the target hashes

4. **Record matches**: When found, the password is successfully cracked

I used two approaches:
- **Local dictionary attack**: Programmatically tested common passwords
- **Online rainbow tables**: Used [CrackStation](https://crackstation.net/), a free service that maintains pre-computed hash databases

In [47]:
# Dictionary attack using common passwords
common_passwords = [
    "password", "123456", "12345678", "qwerty", "abc123", "monkey",
    "letmein", "trustno1", "dragon", "baseball", "iloveyou",
    "P@ssw0rd", "Password1", "admin", "welcome", "cheese"
]

print("Attempting dictionary attack...\n")
found = {}

# Try each password in our dictionary
for password in common_passwords:
    # Encode password as UTF-8 bytes
    h = sha256(password.encode('utf-8'))

    # Check against all target hashes
    for i, target in enumerate(target_hashes):
        if h == target:
            found[i] = password
            print(f"Found password {i+1}: '{password}'")

print(f"\nCracked {len(found)} out of 3 passwords")

Attempting dictionary attack...

Found password 1: 'password'


Found password 3: 'P@ssw0rd'
Found password 2: 'cheese'

Cracked 3 out of 3 passwords


#### Results

The three passwords were successfully cracked:

1. **`password`** - The most commonly used weak password globally
2. **`P@ssw0rd`** - Common variation using character substitutions (@ for 'a', 0 for 'o')
3. **`cheese`** - Simple dictionary word, appears in common password lists

All three passwords appear in the top 1000 most common passwords found in data breaches.

In [50]:
# Verify the actual passwords (found using CrackStation)
actual_passwords = ["password", "cheese", "P@ssw0rd"]

print("Verification of actual passwords:")
for i, (password, target) in enumerate(zip(actual_passwords, target_hashes)):
    computed = sha256(password.encode('utf-8'))
    match = computed == target
    print(f"Password {i+1}: '{password}' -> Match: {match}")

Verification of actual passwords:
Password 1: 'password' -> Match: True
Password 2: 'cheese' -> Match: True
Password 3: 'P@ssw0rd' -> Match: True


#### How the Passwords Were Found

Dictionary Attack Method:
1. Compiled a list of common passwords from various sources
2. Hashed each password using SHA-256
3. Compared each hash against the target hashes
4. When a match was found, the password was recovered

This works because:
1. No salt: Same password always produces the same hash
2. Single-pass SHA-256: Very fast to compute 
3. Common passwords: All three are in common password lists and breach databases

#### Security Improvements

1. Salt: Add random value to each password before hashing
2. KDFs: Use bcrypt, scrypt, or Argon2 instead of plain SHA-256
3. Iterations: Hash multiple times 
4. Pepper: Add secret key to all passwords
5. Strong passwords: Longer passwords, password managers

#### Demonstration of Improved Password Hashing

This example shows how salt and iterations dramatically improve security.

In [49]:
# Demonstration of improved password hashing
def secure_password_hash(password, salt=None, iterations=100000):
    """
    Example of improved password hashing with salt and iterations.
    
    Args:
        password (str): Password to hash
        salt (bytes): Random salt (generated if not provided)
        iterations (int): Number of hash iterations
    
    Returns:
        tuple: (salt_hex, hash_hex) both as hexadecimal strings
    """
   
    # Generate random salt if not provided
    if salt is None:
        salt = os.urandom(16)  # 128-bit salt
    
    # Combine password and salt
    salted = password.encode('utf-8') + salt
    
    # Apply multiple iterations
    current = salted
    for _ in range(iterations):
        current = sha256(current).encode('utf-8')
    
    return salt.hex(), current.decode('utf-8')

# Demonstrate that same password produces different hashes with different salts
password = "password"
salt1, hash1 = secure_password_hash(password)
salt2, hash2 = secure_password_hash(password)

print("Demonstration of salted hashing:")
print(f"Password: '{password}'")
print(f"\nWith salt 1:")
print(f"  Salt: {salt1}")
print(f"  Hash: {hash1[:32]}...")
print(f"\nWith salt 2:")
print(f"  Salt: {salt2}")
print(f"  Hash: {hash2[:32]}...")
print(f"\nHashes are different: {hash1 != hash2}")


Demonstration of salted hashing:
Password: 'password'

With salt 1:
  Salt: 93935e100e0e35bc59a56572bad79884
  Hash: 6f97e1522095186689b42d438bc2a8eb...

With salt 2:
  Salt: c9d8206bba88c6415a333ccba6703ccd
  Hash: cc384d2ed57006bd62a24fe5f4c7d461...

Hashes are different: True


## END